# Langchain Basic Orchestration

In [5]:
# Simple print output function for wrapping response
def jprint(content, wrap_at=90):
    """Prints wrapped text or renders Markdown in Jupyter."""
    try:
        from IPython.display import display, Markdown

        display(Markdown(content))
    except ImportError:
        import textwrap

        print(textwrap.fill(content, width=wrap_at))

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize a Chat Model
chat_model = ChatOpenAI(model="gpt-3.5-turbo")

prompt = PromptTemplate.from_template("Summarize {topic} in one paragraph.")

# Initialize the parser
parser = StrOutputParser()

# Formatting the prompt with a variable
# formatted_prompt = prompt.format(topic="Memory in AI")

### Creating a Basic Chain
A Chain is a sequence of steps that process input one stage at a time. Each step transforms information before passing it forward

In [6]:
# Chains turn reasoning into a structured pipeline
# Here we combine: Prompt -> Model -> Output Parser
chain = prompt | chat_model | parser

response = chain.invoke({"topic": "Chains in LangChain"})
jprint(response)

Chains in LangChain are directed acyclic graphs that represent syntactic structures in natural language. Each node in a chain corresponds to a word or phrase in the sentence, and edges between nodes capture the relationships between them, such as dependencies or semantic roles. Chains provide a compact and structured way to represent linguistic information, making it useful for tasks such as parsing, semantic role labeling, and information extraction.

### Adding Memory
Memory stores information from previous interactions. This enables continuity and context-aware conversations

**NOTE - better to use langgraph now**

In [ ]:
# NOTE: This is the old style, better to use Langgraph now!
# Change the import to use the classic path
from langchain_classic.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input": "My name is Gemini"}, {"output": "Hello Gemini!"})

print(memory.load_memory_variables({}))

{'history': 'Human: My name is Gemini\nAI: Hello Gemini!'}


### Tools and Agents
Tools enable LLMs to take real actions like API calls or database queries. An Agent uses reasoning to decide which tool to use and when.

In LangChain, an Agent acts as a reasoning engine. It doesn't just give one answer; it decides which Tool to use based on your question.

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. Initialize the LLM as a 'Reasoning Engine'
model = ChatOpenAI(model="gpt-3.5-turbo")


# 2. Define a simple Tool (representing real actions)
def get_word_length(word):
    print(f"word in tool is: {word}")
    return len(word)


# 3. Create a 'Decision Logic' Prompt
# Prompts are reusable, modular components
logic_prompt = PromptTemplate.from_template(
    "You are an AI that decides if a tool is needed. "
    "Tool: 'WordLength' (counts letters). "
    "Question: {question}. "
    "Do you need the tool? Answer 'Yes' or 'No'."
)

# 4. Building the Chain (a sequence of steps)
# This connects the prompt, model, and output parser
chain = logic_prompt | model | StrOutputParser()

# 5. Execute and Observe [cite: 50]
# question = "How many letters are in all forms of the word 'Orchestration'?"
question = "What is the exact letter count in the 3rd syllable of the word 'Antidisestablishmentarianism'?"
decision = chain.invoke({"question": question})

print(f"Agent Decision: {decision}")


if "Yes" in decision:
    # The app calls the tool to produce a structured decision
    # result = get_word_length("Orchestration")
    result = get_word_length("Antidisestablishmentarianism")
    print(f"Tool Output: {result}")

Agent Decision: Yes
word in tool is: Antidisestablishmentarianism
Tool Output: 28
